<div style="background-color:#192015; color:#7fa637; padding:12px; border-radius:8px; max-width:80%; width:auto; margin:0 auto;">

![nvmath-python](_assets/nvmath_head_panel@0.25x.png)

<p style="font-size:0.85em; margin-top:8px;">
Copyright (c) 2025, NVIDIA CORPORATION & AFFILIATES<br>
SPDX-License-Identifier: BSD-3-Clause
</p>

</div>

# Getting started with nvmath-python: device APIs

In this tutorial we provide basic 101 about **nvmath-python**, how it fits in and 
plays with an existing scientific computing ecosystem in Python and what makes it 
a useful addition for this ecosystem. 

<div class="alert alert-box alert-info">
    To use this notebook you will need a computer equipped with NVIDIA GPU as well 
    as an environment with properly installed Python libraries and (optionally) 
    CUDA Toolkit. Please refer to the nvmath-python documentation for getting 
    familiar with <a href="https://docs.nvidia.com/cuda/nvmath-python/0.2.1/installation.html#install-nvmath-python">installation options</a>.
</div>

This section demonstrates the use of nvmath-python from within custom kernels 
written using *numba-cuda*. The problem being solved is Monte Carlo stock price 
simulation with *Geometric Brownian Motion* (GBM) model.

But first, let's borrow the benchmarking helper function we created in the previous 
section to continue performance experiments.

In [ ]:
import numpy as np
import cupyx as cpx


# Helper function to benchmark two implementations F and (optionally) F_alternative
# When F_alternative is provided, in addition to raw performance numbers (seconds)
# speedup of F relative to F_alternative is reported
def benchmark(
    F, F_name="Implementation", F_alternative=None, F_alternative_name="Alternative implementation", n_repeat=10, n_warmup=1
):
    # warm-up + repeated runs
    timing = cpx.profiler.benchmark(F, n_repeat=n_repeat, n_warmup=n_warmup)
    # best time from repeated runs
    perf = np.min(timing.gpu_times)
    print(f"{F_name} performance = {perf:0.4f} sec")

    if F_alternative is not None:
        timing_alt = cpx.profiler.benchmark(F_alternative, n_repeat=n_repeat, n_warmup=n_warmup)
        perf_alt = np.min(timing_alt.gpu_times)
        print(f"{F_alternative_name} performance = {perf_alt:0.4f} sec")
        print(f"Speedup = {perf_alt / perf:0.4f}x")
    else:
        perf_alt = None

    return perf, perf_alt

## Arithmetic Brownian motion

*Arithmetic Brownian motion* (which is also often called a *simple Brownian motion*) 
is among the most important stochastic processes, also known as the *Wiener process*.

In its normalized form, the process $B_t$ always starts at $0$ at the moment $t = 0$ 
and evolves to time $t = T$ with zero mean and the standard deviation $\sqrt{T}$. 
Moreover, at any two given time moments $t$ and $s$, where $t>s$, the increment 
$B_t - B_s$ is distributed normally with zero mean and standard deviation $t-s$:

$B_0 = 0$

$B_t - B_s \sim \mathcal{N}{\left(0, t-s\right)}$

Finally, any two increments are independent.


In [ ]:
def brownian_motion(nsteps, npaths):
    dBt = np.random.randn(npaths, nsteps - 1)
    dBt = np.insert(dBt, 0, 0.0, axis=1)  # The process starts at 0
    Bt = np.cumsum(dBt, axis=1)
    return Bt


RNG_SEED = 77777  # Random seed
N_STEPS = 100  # Number of time steps
N_PATHS = 500  # Number of simulated paths

np.random.seed(RNG_SEED)
paths = brownian_motion(N_STEPS, N_PATHS)
t = np.arange(N_STEPS)

Let's visualize how process behaves:

In [ ]:
from matplotlib import pyplot as plt
import math
from scipy.stats import norm

LIGHTGREY = "#AAA"
GREY = "#777"
BLACK = "#000"
BLUE = "#88f"
DARKBLUE = "#55a"

fig = plt.figure(figsize=(13, 6))
ax0 = plt.subplot2grid((1, 4), (0, 0), colspan=3)

ax0.grid(True, linestyle="--")
ax0.axvline(x=0, color="k")
for i in range(1, paths.shape[0]):
    ax0.plot(t, paths[i], color=BLUE, linewidth=1)
ax0.plot(t, paths[0], color=DARKBLUE, linewidth=4)
ax0.plot(t, np.sqrt(t), color=BLACK, linestyle=":", linewidth=2)
ax0.plot(t, -np.sqrt(t), color=BLACK, linestyle=":", linewidth=2)
ax0.plot(t, 2 * np.sqrt(t), color=BLACK, linestyle=":", linewidth=2)
ax0.plot(t, -2 * np.sqrt(t), color=BLACK, linestyle=":", linewidth=2)
ax0.plot(t, 3 * np.sqrt(t), color=BLACK, linestyle=":", linewidth=2)
ax0.plot(t, -3 * np.sqrt(t), color=BLACK, linestyle=":", linewidth=2)
ax0.annotate(r"$\sqrt{t}$", xy=(50, 9), fontsize=16, fontweight="bold", rotation=6, ha="center", va="center")
ax0.annotate(r"$2\sqrt{t}$", xy=(50, 16), fontsize=16, fontweight="bold", rotation=10, ha="center", va="center")
ax0.annotate(r"$3\sqrt{t}$", xy=(50, 24), fontsize=16, fontweight="bold", rotation=12, ha="center", va="center")
ax0.set_xlabel("Time Steps")
ax0.set_title("Brownian motion")
ax0.axis("on")
ax0.set_ylim(-30, 30)

x = np.linspace(-30, 30)
ax1 = plt.subplot2grid((1, 4), (0, 3), colspan=1)
ax1.hist(paths[:, -1], bins=29, density=True, orientation="horizontal", color=BLUE, edgecolor=DARKBLUE)
ax1.plot(norm.pdf(x, loc=0, scale=math.sqrt(N_STEPS)), x, color=BLACK)
ax1.set_ylim(-30, 30)
ax1.get_yaxis().set_visible(False)
ax1.set_title(f"Histogram at $t={N_STEPS}$")
ax1.annotate(r"$\mathcal{N}{\left(0, t\right)}$", xy=(0.04, 0), va="center", rotation=-90, fontsize=16)

plt.show()

## Geometric Brownian motion

It is a derivative process from arithmetic Brownian motion, which is useful in 
simulating non-negative stochastic processes.

$S_t = S_0 \exp \left(\left( \mu - \sigma^2 / 2 \right) t + \sigma B_t \right)$

Here $\mu$ is called the *drift* and $\sigma$ is called the *volatility*. Sometimes 
it is more useful to express the process in the *differential form*:

$dS_t = \mu S_t + \sigma S_t dB_t$

As the first order approximation, stock prices can be often modeled as the geometric 
Brownian motion (GBM). Unlike simple Brownian motion, which starts at $0$, the 
geometric Brownian motions starts at $S_0$ and never goes below $0$.

In [ ]:
S0 = 100.0  # Initial stock price
MU = 0.003
SIGMA = 0.027

RNG_SEED = 77777  # Random seed
N_STEPS = 252  # Number of time steps
N_PATHS = 500  # Number of simulated paths


def brownian_motion(nsteps, npaths, mu, sigma):
    # Differential form of the Brownian motion
    dBt = np.random.randn(npaths, nsteps - 1) * sigma + mu
    dBt = np.insert(dBt, 0, 0.0, axis=1)  # The process starts at 0

    # Integral form of the Brownian motion
    Bt = np.cumsum(dBt, axis=1)

    return Bt


np.random.seed(RNG_SEED)
b_t = brownian_motion(N_STEPS, N_PATHS, MU, SIGMA)
s_t = S0 * np.exp(b_t)

print(f"Mean stock price at t=T: {s_t[:, -1].mean():0.2f}")
print(f"Standard deviation of stock price at t=T: {s_t[:, -1].std():0.2f}")
print(type(s_t))

Let's visualize $S_t$:

In [ ]:
fig = plt.figure(figsize=(13, 6))
ax = plt.subplot2grid((1, 1), (0, 0), colspan=1)
for i in range(s_t.shape[0]):
    ax.plot(s_t[i], color=BLUE, linewidth=1)
ax.set_title(" Stock price simulated as GBM")
plt.show()

Let us create a GPU version of the code using CuPy.

In [ ]:
import nvmath
import cupy as cp

RNG_SEED = 777777  # Random seed
N_STEPS = 252  # Number of time steps (trading days in a year)
N_PATHS = 800000  # Number of simulated paths (large number to get a reliable estimate)

S0 = 100.0  # Initial stock price
MU = 0.003  # Drift with upward trend
SIGMA = 0.027  # Volatility


def brownian_motion(nsteps, npaths, mu, sigma):
    # Differential form of the Brownian motion
    dBt = cp.empty((npaths, nsteps), dtype=cp.float32, order="F")
    dBt[:, 0] = 0.0  # The process starts at 0
    dBt[:, 1:] = cp.random.randn(npaths, nsteps - 1) * sigma + mu

    # Integral form of the Brownian motion
    Bt = cp.cumsum(dBt, axis=1)

    return Bt


def generate_gbm_paths_cupy(npaths, nsteps, mu, sigma, s0):
    b_t = brownian_motion(nsteps, npaths, mu, sigma)
    paths = s0 * cp.exp(b_t)
    return paths


cp.random.seed(RNG_SEED)
s_t = generate_gbm_paths_cupy(N_PATHS, N_STEPS, MU, SIGMA, S0)

print(f"Mean stock price at t=T: {s_t[:, -1].mean():0.2f}")
print(f"Standard deviation of stock price at t=T: {s_t[:, -1].std():0.2f}")
print(type(s_t))

The above code consists of a chain of primitive operations with very low arithmetic 
intensity. Each primitive operation processes an array of the shape `(N_PATHS, N_STEPS)`, 
which makes the overall workload heavily memory-bound.

One of ways to improve the workload's arithmetic intensity and to make it less 
memory-bound is to implement a custom device kernel using *numba-cuda*, which will 
handle the entire Monte Carlo path from $t=0$ till $t=T$ by an individual thread. 
The total number of threads will be equal to `N_PATHS`.

The nvmath-python provides device APIs for *random number generation*, which can be 
used within *numba-cuda* kernel for effective random number generation. The following 
code illustrates the implementation of the GBM using *numba-cuda* and nvmath-python's 
random number generator.

In [ ]:
from numba import cuda
from nvmath.device import random
import cupy as cp

RNG_SEED = 777777  # Random seed
N_STEPS = 252  # Number of time steps (trading days in a year)
N_PATHS = 800000  # Number of simulated paths (large number to get a reliable estimate)

S0 = 100.0  # Initial stock price
MU = 0.003  # Drift with upward trend
SIGMA = 0.027  # Volatility

# Pre-compile the random number generator into IR to use alongside other device code
compiled_rng = random.Compile(cc=None)

# Set up CUDA kernel launch configuration
threads_per_block = 32
blocks = N_PATHS // threads_per_block
nthreads = threads_per_block * blocks
print(f"blocks: {blocks}, threads_per_block: {threads_per_block}, nthreads: {nthreads}")

# Allocate space for random states
states = random.StatesPhilox4_32_10(nthreads)


# RNG initialization kernel
@cuda.jit(link=compiled_rng.files, extensions=compiled_rng.extension)
def init_rng_gpu(states, seed):
    idx = cuda.grid(1)
    random.init(seed, idx, 0, states[idx])


# GBM path generation kernel. Note that the random numbers are generated
# as they are needed, unlike for the NumPy/CuPy implementation where they are
# generated upfront and stored.
@cuda.jit(link=compiled_rng.files, extensions=compiled_rng.extension)
def generate_gbm_paths_nvmath(states, paths, nsteps, mu, sigma, s0):
    mu = paths.dtype.type(mu)
    sigma = paths.dtype.type(sigma)
    s0 = paths.dtype.type(s0)

    idx = cuda.grid(1)
    if idx >= paths.shape[0]:
        return

    # Each thread generates one path in the time domain
    paths[idx, 0] = s0

    # Consume a single normal variate at a time
    for i in range(1, nsteps):
        z = random.normal(states[idx])
        z = mu + sigma * z
        paths[idx, i] = paths[idx, i - 1] * math.exp(z)


# Allocate space for paths
paths_gpu = cp.empty((N_PATHS, N_STEPS), dtype=cp.float32, order="F")

# Initialize RNG states
init_rng_gpu[blocks, threads_per_block](states, RNG_SEED)

# Generate GBM paths on GPU
generate_gbm_paths_nvmath[blocks, threads_per_block](states, paths_gpu, N_STEPS, MU, SIGMA, S0)

print(f"Mean stock price at t=T: {paths_gpu[:, -1].mean():0.2f}")
print(f"Standard deviation of stock price at t=T: {paths_gpu[:, -1].std():0.2f}")

Now, let's benchmark the nvmath-python implementation and compare to CuPy implementation.

In [ ]:
benchmark(
    lambda: generate_gbm_paths_nvmath[blocks, threads_per_block](states, paths_gpu, N_STEPS, MU, SIGMA, S0),
    "nvmath-python",
    lambda: generate_gbm_paths_cupy(N_PATHS, N_STEPS, MU, SIGMA, S0),
    "CuPy",
    n_repeat=5,
    n_warmup=1,
)

Now let us modify the generation kernel to leverage the fact that the Philox4_32_10 
generator returns 4 variates at a time allowing to produce 4 time steps in a single 
loop iteration.

In [ ]:
from numba import cuda
from nvmath.device import random
import cupy as cp

RNG_SEED = 777777  # Random seed
N_STEPS = 252  # Number of time steps (trading days in a year)
N_PATHS = 800000  # Number of simulated paths (large number to get a reliable estimate)

S0 = 100.0  # Initial stock price
MU = 0.003  # Drift with upward trend
SIGMA = 0.027  # Volatility

# Pre-compile the random number generator into IR to use alongside other device code
compiled_rng = random.Compile(cc=None)

# Set up CUDA kernel launch configuration
threads_per_block = 32
blocks = N_PATHS // threads_per_block
nthreads = threads_per_block * blocks
print(f"blocks: {blocks}, threads_per_block: {threads_per_block}, nthreads: {nthreads}")

# Allocate space for random states
states = random.StatesPhilox4_32_10(nthreads)


# RNG initialization kernel
@cuda.jit(link=compiled_rng.files, extensions=compiled_rng.extension)
def init_rng_gpu(states, seed):
    idx = cuda.grid(1)
    random.init(seed, idx, 0, states[idx])


@cuda.jit(link=compiled_rng.files, extensions=compiled_rng.extension)
def generate_gbm_paths_gpu(states, paths, nsteps, mu, sigma, s0):
    mu = paths.dtype.type(mu)
    sigma = paths.dtype.type(sigma)
    s0 = paths.dtype.type(s0)

    idx = cuda.grid(1)
    if idx >= paths.shape[0]:
        return

    # Each thread generates one path in the time domain
    paths[idx, 0] = s0

    # Consume 4 normal variates at a time for better throughput
    for i in range(1, nsteps, 4):
        v = random.normal4(states[idx])  # Returned as float32x4 type
        vals = v.x, v.y, v.z, v.w  # Decompose into a tuple of float32
        # Process a chunk of 4 time steps, use min() to avoid out-of-bounds access
        for j in range(i, min(i + 4, nsteps)):
            paths[idx, j] = paths[idx, j - 1] * math.exp(mu + sigma * vals[j - i])


# Allocate space for paths
paths_gpu = cp.empty((N_PATHS, N_STEPS), dtype=cp.float32, order="F")

# Initialize RNG states
init_rng_gpu[blocks, threads_per_block](states, RNG_SEED)

# Generate GBM paths on GPU
generate_gbm_paths_nvmath[blocks, threads_per_block](states, paths_gpu, N_STEPS, MU, SIGMA, S0)

print(f"Mean stock price at t=T: {paths_gpu[:, -1].mean():0.2f}")
print(f"Standard deviation of stock price at t=T: {paths_gpu[:, -1].std():0.2f}")

In [ ]:
benchmark(
    lambda: generate_gbm_paths_nvmath[blocks, threads_per_block](states, paths_gpu, N_STEPS, MU, SIGMA, S0),
    "nvmath-python",
    lambda: generate_gbm_paths_cupy(N_PATHS, N_STEPS, MU, SIGMA, S0),
    "CuPy",
    n_repeat=5,
    n_warmup=1,
)

<div style="background-color:#76B900; color:#ffffff; padding:12px; border-radius:8px; max-width:80%; width:auto; margin:0 auto;">
<h3 style="margin-top:0; color:#ffffff">Takeaways</h3>

- nvmath-python device APIs allow direct integration within custom numba-cuda kernels, dramatically reducing implementation costs of math-intense kernels.
- Kernel fusion eliminates intermediate array allocations and memory transfers, significantly improving overall arithmetic intensity.
</div>